In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/")

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
X_batch, y_batch = mnist.train.next_batch(1)

In [3]:
tf.reset_default_graph()

input_dim = [28,28]

num_classes = 10
batch_size = 50
x = tf.placeholder(shape=[batch_size, input_dim[0],input_dim[1], 1], dtype=tf.float32, name="X")
y = tf.placeholder(shape=[batch_size], dtype=tf.int64, name="y")

conv1 = tf.layers.conv2d(x, name="conv1", filters = 64, 
                         kernel_size = 4,strides = 1,padding = "valid", activation = tf.nn.relu)
conv2 = tf.layers.conv2d(conv1, name="conv2", filters = 128, 
                         kernel_size = 4,strides = 1,padding = "valid", activation = tf.nn.relu)
pool1 = tf.layers.max_pooling2d(conv2, pool_size = [2,2], strides = 2)

conv3 = tf.layers.conv2d(pool1, name="conv3", filters = 32, 
                         kernel_size = 4,strides = 1,padding = "valid", activation = tf.nn.relu)
conv4 = tf.layers.conv2d(conv3, name="conv4", filters = 64, 
                         kernel_size = 4,strides = 1,padding = "valid", activation = tf.nn.relu)
pool2 = tf.layers.max_pooling2d(conv4, pool_size = [2,2], strides = 2)
pool2_flat = tf.reshape(pool1,[tf.shape(pool1)[0],pool1.shape[1]*pool1.shape[2]*pool1.shape[3]])




 
####dynamic filter#######
x_flat = tf.reshape(x,[tf.shape(x)[0],x.shape[1]*x.shape[2]*x.shape[3]],name="x_reshape")
d_dense = tf.layers.dense(x_flat,pool2_flat.shape[1]*num_classes, name = "d_dense1")   

W = tf.reshape(d_dense,[tf.shape(x)[0], pool2_flat.shape[1], num_classes],name = "w_reshape")

    

matmul = [tf.matmul(tf.reshape(tf.unstack(pool2_flat)[i],[1,pool2_flat.shape[1]]), 
                    tf.unstack(W)[i]) 
          for i in range(batch_size)]
logits = tf.squeeze(tf.stack(matmul),axis=1)


#W_ = tf.get_variable('W_', [pool2_flat.shape[1], num_classes])
#logits_ = tf.matmul(pool2_flat, W_, name="logits")




In [5]:
predictions = tf.argmax(logits,axis=-1)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.one_hot(y,num_classes), logits=logits))
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [6]:
batch_size = 50
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for i in range(5000):
        X_batch, y_batch = mnist.train.next_batch(batch_size)
        X_batch = X_batch.reshape([-1, 28, 28, 1])

        _,loss_,pred = sess.run([train_op,loss,predictions],feed_dict = {x : X_batch, y : y_batch})
        if i % 50 == 0:
            print("Iteration : {}  Loss : {}  Real : {}  Pred : {}".format(i,loss_,y_batch[0:3],pred[0:3]))

Iteration : 0  Loss : 2.3795158863067627  Real : [3 8 6]  Pred : [7 6 7]
Iteration : 50  Loss : 0.11844810843467712  Real : [1 0 4]  Pred : [1 0 9]
Iteration : 100  Loss : 0.14871624112129211  Real : [9 7 3]  Pred : [9 7 3]
Iteration : 150  Loss : 0.02912949025630951  Real : [7 5 4]  Pred : [7 5 4]
Iteration : 200  Loss : 0.11489465832710266  Real : [1 7 8]  Pred : [1 7 8]
Iteration : 250  Loss : 0.09161071479320526  Real : [0 6 1]  Pred : [0 6 1]
Iteration : 300  Loss : 0.05241715535521507  Real : [3 2 9]  Pred : [3 2 4]
Iteration : 350  Loss : 0.025655724108219147  Real : [8 0 6]  Pred : [8 0 6]
Iteration : 400  Loss : 0.12459543347358704  Real : [0 1 2]  Pred : [0 1 2]
Iteration : 450  Loss : 0.07976213097572327  Real : [4 4 3]  Pred : [4 4 3]
Iteration : 500  Loss : 0.0438171923160553  Real : [1 9 4]  Pred : [1 9 4]
Iteration : 550  Loss : 0.1925077885389328  Real : [2 6 0]  Pred : [2 5 0]
Iteration : 600  Loss : 0.022128403186798096  Real : [4 1 0]  Pred : [4 1 0]
Iteration : 650 

KeyboardInterrupt: 